In [1]:
import torch.nn as nn
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import matplotlib.pyplot as plt 
import numpy as np
import sys

In [2]:
import sys
sys.path.append("../LAB1/")  # Add the directory to sys.path

import resnet  # Now you can import resnet.py
from resnet import ResNet18

In [3]:
# # We load the dictionary
loaded_cpt = torch.load('../LAB1/model_state/resnet18.pth')

model = ResNet18()
model.load_state_dict(loaded_cpt['model_state'])

/tmp/ipykernel_341049/1518593490.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_cpt = torch.load('../LAB1/model_state/resnet18.pth')


<All keys matched successfully>

## Load the dataset

In [4]:
batch_size = 64 

In [5]:
## Normalization adapted for CIFAR10
normalize_scratch = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

# Transforms is a list of transformations applied on the 'raw' dataset before the data is fed to the network.
# Here, Data augmentation (RandomCrop and Horizontal Flip) are applied to each batch, differently at each epoch, on the training set data only
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    normalize_scratch,
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_scratch,
])

### The data from CIFAR10 are already downloaded in the following folder
rootdir = '/opt/img/effdl-cifar10/'

c10train = CIFAR10(rootdir,train=True,download=True,transform=transform_train)
c10test = CIFAR10(rootdir,train=False,download=True,transform=transform_test)

trainloader = DataLoader(c10train,batch_size=batch_size,shuffle=True)
testloader = DataLoader(c10test,batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU (if available) and convert to FP16
model.to(device).half()
model.eval()

correct = 0
total = 0

with torch.no_grad():  # Disable gradient computation for inference
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device).half(), labels.to(device)  # Convert inputs to FP16, keep labels in int format
        
        outputs = model(inputs)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted class index
        
        total += labels.size(0)  # Total number of samples
        correct += (predicted == labels).sum().item()  # Count correct predictions

# Compute accuracy
accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 90.32%


In [ ]:
import binaryconnect
    ### define your model somewhere, let's say it is called "mymodel"

mymodelbc = binaryconnect.BC(mymodel) ### use this to prepare your model for binarization
mymodelbc.model = mymodelbc.model.to(device) # it has to be set for GPU training
### During training (check the algorithm in the course and in the paper to see the exact sequence of operations)
mymodelbc.binarization() ## This binarizes all weights in the model
mymodelbc.restore() ###  This reloads the full precision weights
### After backprop
mymodelbc.clip() ## Clip the weights